In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from models.xswin import XNetSwinTransformer
from models.modules import SwinResidualCrossAttention

from models.modules.swin_residual_cross_attention import _extract_windows, _unfold_padding_prep, _fold_unpadding_prep

from torchinfo import summary


In [2]:
*A, B= 1, 2
A


[1]

In [3]:
x = torch.tensor([
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
])

x = torch.tensor([
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
])

# print(torch.cat((x, x), dim=-1))


# x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))
x = torch.stack((x, x*2, x*3, x*4))
x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))


# x = x.reshape(2, 2, -1)
x = x.unsqueeze(0)

print(x.shape)

x = _extract_windows(x, 4, 4)

# print(x.shape)


torch.Size([1, 8, 8, 5])


In [4]:
H_DIM = 16

r1 = torch.ones((5, 10, 22, H_DIM))
r2 = torch.ones(r1.shape)

f1, pinfo = _unfold_padding_prep(r1, 4, 4)
print(f1[0, :, :0, 0])
f2 = _fold_unpadding_prep(f1, pinfo)
print(f2.shape)

SRCA = SwinResidualCrossAttention(
    [4, 4],
    H_DIM,
    4,
)

SRCA(r1, r2).shape


tensor([], size=(12, 0))
torch.Size([5, 10, 22, 16])


torch.Size([5, 10, 22, 16])

In [5]:
patch_size = [4, 4]
embed_dim = 64
depths = [3, 3, 3]
num_heads = [4, 8, 16]
window_size = [4, 4]
num_classes = 1


swin = XNetSwinTransformer(patch_size, embed_dim, depths, 
                           num_heads, window_size, num_classes=num_classes,
                           final_downsample=False, residual_cross_attention=True,
                           )

IMG_H, IMG_W = 164, 328
x = torch.randn((5, 3, IMG_H, IMG_W))
print(x.shape)

# print(swin)

y = swin(x)
print(y.shape)

# print(swin)

summary(swin, input_size=[1, 3, IMG_H, IMG_W])


torch.Size([5, 3, 164, 328])
torch.Size([5, 164, 328])


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                                  Output Shape              Param #
XNetSwinTransformer                                     [1, 164, 328]             --
├─ConvolutionTripletLayer: 1-1                          [1, 64, 164, 328]         --
│    └─Sequential: 2-1                                  [1, 64, 164, 328]         --
│    │    └─Conv2d: 3-1                                 [1, 64, 164, 328]         1,792
│    │    └─BatchNorm2d: 3-2                            [1, 64, 164, 328]         128
│    │    └─LeakyReLU: 3-3                              [1, 64, 164, 328]         --
│    │    └─Conv2d: 3-4                                 [1, 64, 164, 328]         36,928
│    │    └─BatchNorm2d: 3-5                            [1, 64, 164, 328]         128
│    │    └─LeakyReLU: 3-6                              [1, 64, 164, 328]         --
│    │    └─Conv2d: 3-7                                 [1, 64, 164, 328]         36,928
│    │    └─BatchNorm2d: 3-8                   